In [1]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import random
import gensim as gs
import warnings
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
warnings.filterwarnings('ignore')
from sklearn.metrics import pairwise_distances
from heapq import nsmallest

In [2]:
test_articles = pd.read_csv('../dataset/test_articles_newyork.csv', delimiter=',')

In [10]:
fixed_index = pd.read_csv('../dataset/fixed_test_set_newyork_max_recall5.csv', delimiter=',')

In [4]:
from bert_serving.client import BertClient
bc = BertClient()

In [5]:
def get_accuracy(ID_goal,ranked_tables_ID):
    
    accuracy = 0
    
    for table_ID in ranked_tables_ID:
        
        if table_ID[0] == ID_goal:
    
            accuracy = 1
            break;

    return accuracy

In [6]:
def get_mrr(ID_goal,ranked_tables_ID):
    
    accuracy = 0
    index_match = 1

    for idTable in ranked_tables_ID:
        
        if idTable[0] == ID_goal:
    
            accuracy = 1/index_match
            break;
        
        index_match = index_match + 1
   
    return accuracy

In [7]:
test_articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148 entries, 0 to 147
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   article_full_text         141 non-null    object
 1   article_key_match         148 non-null    object
 2   article_keywords          143 non-null    object
 3   article_main_passage      148 non-null    object
 4   article_meta_description  132 non-null    object
 5   article_meta_keywords     58 non-null     object
 6   article_summary           141 non-null    object
 7   article_tags              54 non-null     object
 8   article_title             148 non-null    object
 9   article_url               148 non-null    object
dtypes: object(10)
memory usage: 11.7+ KB


In [8]:
fixed_index.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14800 entries, 0 to 14799
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   label_index              14800 non-null  object
 1   table_page_id            14800 non-null  object
 2   table_page_title         14800 non-null  object
 3   table_page_main_passage  14149 non-null  object
 4   table_page_keywords      10950 non-null  object
dtypes: object(5)
memory usage: 578.2+ KB


In [11]:
evaluate_topk = [1,5,10,20,50]

result = []

#evaluating each topk value
for TOP_K in evaluate_topk:

    accuracy = []
    mrr = []

    for i, row in tqdm(test_articles.iterrows()):

        #current article values
        article_ID = row['article_key_match']
        article_title_text = str(row['article_title'])
        article_title_main_passage_text = str(row['article_meta_description'])

        #embedding and model variables
        article_title = []
        title_table = []
        ranked_tables_model = []

        #return index
        return_index = fixed_index.loc[fixed_index['label_index'] == row['article_key_match']]
        
        article_title.append(article_title_text+" "+article_title_main_passage_text)
        embedding_articles = bc.encode(article_title)
        article_dense_vector = []
        for current_embedding in embedding_articles:
            article_dense_vector.append(current_embedding)

        #creating embedding 
        for i, row in return_index.iterrows():
            
            title_table.append(str(row['table_page_title'])+" "+str(row['table_page_main_passage']))

        embedding_tables = bc.encode(title_table)
        tables_dense_vector = []
        for current_embedding in embedding_tables:
            tables_dense_vector.append(current_embedding)
        
        distance_vector = pairwise_distances(article_dense_vector, tables_dense_vector, metric='cosine')
        
        #creating the final dataframe
        for i in range(0,len(distance_vector[0])):

            ranked_tables_model.append([return_index.iloc[i]['table_page_id'],return_index.iloc[i]['table_page_title'],distance_vector[0][i]]) 

        data_frame = pd.DataFrame(ranked_tables_model, columns = ['table_ID', 'table_title','table_ranking']) 
        data_frame_sorting = data_frame.sort_values('table_ranking')
        
        selected_top = data_frame_sorting.head(TOP_K)
#         max_score = selected_top['table_ranking'].max()
#         draw_tables_socres = data_frame_sorting[data_frame_sorting['table_ranking'] <= max_score]
        final_ranked_tables = selected_top.iloc[:,0:1].values

        #getting topk accuracy
        accuracy.append(get_accuracy(article_ID, final_ranked_tables))

        #testing mean reciprocal rank at k = 50
        if TOP_K == 50:

            mrr.append(get_mrr(article_ID, final_ranked_tables))

    result.append(["Acc@"+str(TOP_K),str(round(np.mean(accuracy),4))])

print("")
print(result[0])
print(result[1])
print(result[2])
print(result[3])
print(result[4])
print("MRR: "+str(round(np.mean(mrr),4)) )
print(mrr)
print("")

148it [12:40,  5.14s/it]
148it [12:35,  5.11s/it]
148it [12:35,  5.11s/it]
148it [12:35,  5.11s/it]
148it [12:40,  5.14s/it]


['Acc@1', '0.2838']
['Acc@5', '0.5203']
['Acc@10', '0.6351']
['Acc@20', '0.7568']
['Acc@50', '0.8986']
MRR: 0.4016
[0, 0.125, 1.0, 0.14285714285714285, 0, 0.023255813953488372, 0, 0.025, 0.5, 1.0, 1.0, 0.5, 0.02, 1.0, 0.03125, 1.0, 0, 0.02857142857142857, 1.0, 1.0, 0.125, 0.058823529411764705, 1.0, 0.14285714285714285, 0.1111111111111111, 0.25, 0.3333333333333333, 0.3333333333333333, 0.5, 0.25, 1.0, 0.0625, 0.03225806451612903, 0.07692307692307693, 0.3333333333333333, 0.2, 1.0, 1.0, 0.04, 0.02857142857142857, 0.3333333333333333, 1.0, 0.125, 0.3333333333333333, 1.0, 0, 1.0, 0.05555555555555555, 0.037037037037037035, 0.2, 1.0, 0.058823529411764705, 1.0, 0.03125, 1.0, 0.25, 0, 0, 0.5, 0.14285714285714285, 0.03125, 0.5, 0.25, 1.0, 1.0, 1.0, 0.02127659574468085, 1.0, 1.0, 1.0, 0, 1.0, 0.5, 0.5, 0, 0.07692307692307693, 1.0, 0.25, 0, 0.14285714285714285, 0.5, 1.0, 0.14285714285714285, 0, 0.08333333333333333, 0.1, 1.0, 0.5, 1.0, 0.047619047619047616, 1.0, 0.14285714285714285, 0.33333333333333